Contains checkpoint 1234 but with small sample sizes.

Used to check for errors in the pipeline.

# Pre-experiment

## Machine Preparation

In [ ]:
%%bash
pwd
ls -a
du ~ -hs
date

In [ ]:
!sudo add-apt-repository ppa:deadsnakes/ppa -y > /dev/null 2>&1

In [ ]:
%%bash
sudo apt -qq update -y
sudo apt -qq upgrade -y

In [ ]:
!sudo DEBIAN_FRONTEND=noninteractive apt install python3.9 python3.9-venv unzip -yqq

In [ ]:
!mkdir artifact_checkgpt

In [ ]:
!python3.9 -m venv ./artifact_checkgpt/.venv

In [ ]:
%%bash
wget -nv https://zenodo.org/records/11215222/files/CheckGPT.zip?download=1 -O ./artifact_checkgpt/CheckGPT.zip
wget -nv https://zenodo.org/records/11215222/files/CheckGPT_presaved_files.zip?download=1 -O ./artifact_checkgpt/CheckGPT_presaved_files.zip
wget -nv https://zenodo.org/records/11215222/files/CS.zip?download=1 -O ./artifact_checkgpt/CS.zip
wget -nv https://zenodo.org/records/11215222/files/PHX.zip?download=1 -O ./artifact_checkgpt/PHX.zip
wget -nv https://zenodo.org/records/11215222/files/HSS.zip?download=1 -O ./artifact_checkgpt/HSS.zip
wget -nv https://zenodo.org/records/11215222/files/GPABenchmark.zip?download=1 -O ./artifact_checkgpt/GPABenchmark.zip
wget -nv https://zenodo.org/records/11215222/files/scripts.zip?download=1 -O ./artifact_checkgpt/scripts.zip
wget -nv https://zenodo.org/records/11215222/files/README.md?download=1 -O ./artifact_checkgpt/README.md
wget -nv https://zenodo.org/records/11215222/files/requirements.txt?download=1 -O ./artifact_checkgpt/requirements.txt

In [ ]:
%%bash
unzip -q ./artifact_checkgpt/CheckGPT.zip -d ./artifact_checkgpt/
unzip -q ./artifact_checkgpt/CheckGPT_presaved_files.zip -d ./artifact_checkgpt/
mkdir ./artifact_checkgpt/GPABench2/
unzip -q ./artifact_checkgpt/CS.zip -d ./artifact_checkgpt/GPABench2/
unzip -q ./artifact_checkgpt/PHX.zip -d ./artifact_checkgpt/GPABench2/
unzip -q ./artifact_checkgpt/HSS.zip -d ./artifact_checkgpt/GPABench2/
unzip -q ./artifact_checkgpt/GPABenchmark.zip -d ./artifact_checkgpt/
unzip -q ./artifact_checkgpt/scripts.zip -d ./artifact_checkgpt/CheckGPT/

In [ ]:
%%bash
rm ./artifact_checkgpt/CheckGPT.zip
rm ./artifact_checkgpt/CheckGPT_presaved_files.zip
rm ./artifact_checkgpt/CS.zip
rm ./artifact_checkgpt/PHX.zip
rm ./artifact_checkgpt/HSS.zip
rm ./artifact_checkgpt/GPABenchmark.zip
rm ./artifact_checkgpt/scripts.zip

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate

python --version

pip install -r ./artifact_checkgpt/requirements.txt -q
pip install datetime -q
python -m nltk.downloader punkt

## Source Code Modification

Adjustment to `dnn.py` needed to run experiment on CNN classification head.

In [ ]:
%%bash
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/
sed -i 's/if MODELID == 14:/if MODELID in [4]:/' dnn.py

Adjustment to `dnn.py` needed to run experiment on unified classifiers with a custom sample size.

In [ ]:
%%bash
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

sed -i 's/domain_list = \["CS", "PHY", "LIT"\]/domain_list = ["CS", "PHX", "HSS"]/' dnn.py

In [ ]:
%%bash
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

sed -i "/parser.add_argument('expid', type=str)/a \
parser.add_argument('--sample', type=int, default=50000)" dnn.py

In [ ]:
%%bash
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

sed -i "/ID = args.expid/a \
SAMPLE_SIZE = args.sample" dnn.py

In [ ]:
%%bash
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

sed -i 's/domain_list = \["CS", "PHX", "HSS"\]/domain_list = ["CS", "PHX", "HSS"]\n\t\tif args.domain == f"ALL_{SAMPLE_SIZE}":\n\t\t\tdomain_list = \[f"CS_{SAMPLE_SIZE}", f"PHX_{SAMPLE_SIZE}", f"HSS_{SAMPLE_SIZE}"\]/' dnn.py

In [ ]:
%%bash
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

sed -i $'s/\t/    /g' dnn.py

Adjustment to `dnn.py` needed to run experiment on prompt transferability.

In [1]:
%%bash
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

sed -i 's/domain_list, task_list, prompt_list = \[domain\], \[task\], \[prompt\]/domain_list, task_list, prompt_list = \[domain\], \[task\], \[int(d) for d in str(prompt)\]/' dnn.py

## Extra Logic

In [ ]:
import datetime

def now():
    time = datetime.datetime.now().timestamp()
    return time

def elapsed_time(start_time: float):
    HIS = 3600
    MIS = 60
    end_time = datetime.datetime.now().timestamp()
    elapsed_seconds = end_time - start_time
    elapsed_hours = int(elapsed_seconds / HIS)
    elapsed_seconds = elapsed_seconds - (elapsed_hours * HIS)
    elapsed_minutes = int(elapsed_seconds / MIS)
    elapsed_seconds = int(elapsed_seconds - (elapsed_minutes * MIS))
    elapsed_time = f"{elapsed_hours} hour(s) {elapsed_minutes} minute(s) {elapsed_seconds} second(s)"
    print(elapsed_time)

# Experiments

Reproduction's working directory will always be `~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/` unless specified otherwise through comments.

## CS - 10

In [ ]:
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python features.py CS 1 1 --gpt 0 --number 10
python features.py CS 2 1 --gpt 0 --number 10

python features.py CS 1 1 --gpt 1 --number 10
python features.py CS 1 2 --gpt 1 --number 10
python features.py CS 1 3 --gpt 1 --number 10
python features.py CS 1 4 --gpt 1 --number 10

python features.py CS 2 1 --gpt 1 --number 10
python features.py CS 2 2 --gpt 1 --number 10
python features.py CS 2 3 --gpt 1 --number 10
python features.py CS 2 4 --gpt 1 --number 10

python features.py CS 3 1 --gpt 1 --number 10
python features.py CS 3 2 --gpt 1 --number 10
python features.py CS 3 3 --gpt 1 --number 10
python features.py CS 3 4 --gpt 1 --number 10

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - CheckGPT

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 1 CS_CheckGPTArch_Train_Task1_Prompt1 --batchsize 256
python dnn.py CS_10 1 2 CS_CheckGPTArch_Train_Task1_Prompt2 --batchsize 256
python dnn.py CS_10 1 3 CS_CheckGPTArch_Train_Task1_Prompt3 --batchsize 256
python dnn.py CS_10 1 4 CS_CheckGPTArch_Train_Task1_Prompt4 --batchsize 256

python dnn.py CS_10 2 1 CS_CheckGPTArch_Train_Task2_Prompt1 --batchsize 256
python dnn.py CS_10 2 2 CS_CheckGPTArch_Train_Task2_Prompt2 --batchsize 256
python dnn.py CS_10 2 3 CS_CheckGPTArch_Train_Task2_Prompt3 --batchsize 256
python dnn.py CS_10 2 4 CS_CheckGPTArch_Train_Task2_Prompt4 --batchsize 256

python dnn.py CS_10 3 1 CS_CheckGPTArch_Train_Task3_Prompt1 --batchsize 256
python dnn.py CS_10 3 2 CS_CheckGPTArch_Train_Task3_Prompt2 --batchsize 256
python dnn.py CS_10 3 3 CS_CheckGPTArch_Train_Task3_Prompt3 --batchsize 256
python dnn.py CS_10 3 4 CS_CheckGPTArch_Train_Task3_Prompt4 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 1 CS_CheckGPTArch_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task1_Prompt1/Best_CS_10_Task1.pth
python dnn.py CS_10 1 2 CS_CheckGPTArch_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task1_Prompt2/Best_CS_10_Task1.pth
python dnn.py CS_10 1 3 CS_CheckGPTArch_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task1_Prompt3/Best_CS_10_Task1.pth
python dnn.py CS_10 1 4 CS_CheckGPTArch_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task1_Prompt4/Best_CS_10_Task1.pth

python dnn.py CS_10 2 1 CS_CheckGPTArch_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task2_Prompt1/Best_CS_10_Task2.pth
python dnn.py CS_10 2 2 CS_CheckGPTArch_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task2_Prompt2/Best_CS_10_Task2.pth
python dnn.py CS_10 2 3 CS_CheckGPTArch_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task2_Prompt3/Best_CS_10_Task2.pth
python dnn.py CS_10 2 4 CS_CheckGPTArch_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task2_Prompt4/Best_CS_10_Task2.pth

python dnn.py CS_10 3 1 CS_CheckGPTArch_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task3_Prompt1/Best_CS_10_Task3.pth
python dnn.py CS_10 3 2 CS_CheckGPTArch_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task3_Prompt2/Best_CS_10_Task3.pth
python dnn.py CS_10 3 3 CS_CheckGPTArch_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task3_Prompt3/Best_CS_10_Task3.pth
python dnn.py CS_10 3 4 CS_CheckGPTArch_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task3_Prompt4/Best_CS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - RCH

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 1 CS_RCH_Train_Task1_Prompt1 --batchsize 256 --modelid 2
python dnn.py CS_10 1 2 CS_RCH_Train_Task1_Prompt2 --batchsize 256 --modelid 2
python dnn.py CS_10 1 3 CS_RCH_Train_Task1_Prompt3 --batchsize 256 --modelid 2
python dnn.py CS_10 1 4 CS_RCH_Train_Task1_Prompt4 --batchsize 256 --modelid 2

python dnn.py CS_10 2 1 CS_RCH_Train_Task2_Prompt1 --batchsize 256 --modelid 2
python dnn.py CS_10 2 2 CS_RCH_Train_Task2_Prompt2 --batchsize 256 --modelid 2
python dnn.py CS_10 2 3 CS_RCH_Train_Task2_Prompt3 --batchsize 256 --modelid 2
python dnn.py CS_10 2 4 CS_RCH_Train_Task2_Prompt4 --batchsize 256 --modelid 2

python dnn.py CS_10 3 1 CS_RCH_Train_Task3_Prompt1 --batchsize 256 --modelid 2
python dnn.py CS_10 3 2 CS_RCH_Train_Task3_Prompt2 --batchsize 256 --modelid 2
python dnn.py CS_10 3 3 CS_RCH_Train_Task3_Prompt3 --batchsize 256 --modelid 2
python dnn.py CS_10 3 4 CS_RCH_Train_Task3_Prompt4 --batchsize 256 --modelid 2

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 1 CS_RCH_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/CS_RCH_Train_Task1_Prompt1/Best_CS_10_Task1.pth
python dnn.py CS_10 1 2 CS_RCH_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/CS_RCH_Train_Task1_Prompt2/Best_CS_10_Task1.pth
python dnn.py CS_10 1 3 CS_RCH_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/CS_RCH_Train_Task1_Prompt3/Best_CS_10_Task1.pth
python dnn.py CS_10 1 4 CS_RCH_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/CS_RCH_Train_Task1_Prompt4/Best_CS_10_Task1.pth

python dnn.py CS_10 2 1 CS_RCH_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/CS_RCH_Train_Task2_Prompt1/Best_CS_10_Task2.pth
python dnn.py CS_10 2 2 CS_RCH_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/CS_RCH_Train_Task2_Prompt2/Best_CS_10_Task2.pth
python dnn.py CS_10 2 3 CS_RCH_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/CS_RCH_Train_Task2_Prompt3/Best_CS_10_Task2.pth
python dnn.py CS_10 2 4 CS_RCH_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/CS_RCH_Train_Task2_Prompt4/Best_CS_10_Task2.pth

python dnn.py CS_10 3 1 CS_RCH_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/CS_RCH_Train_Task3_Prompt1/Best_CS_10_Task3.pth
python dnn.py CS_10 3 2 CS_RCH_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/CS_RCH_Train_Task3_Prompt2/Best_CS_10_Task3.pth
python dnn.py CS_10 3 3 CS_RCH_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/CS_RCH_Train_Task3_Prompt3/Best_CS_10_Task3.pth
python dnn.py CS_10 3 4 CS_RCH_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/CS_RCH_Train_Task3_Prompt4/Best_CS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - MLP-Pool

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 1 CS_MLP_Train_Task1_Prompt1 --batchsize 256 --modelid 3
python dnn.py CS_10 1 2 CS_MLP_Train_Task1_Prompt2 --batchsize 256 --modelid 3
python dnn.py CS_10 1 3 CS_MLP_Train_Task1_Prompt3 --batchsize 256 --modelid 3
python dnn.py CS_10 1 4 CS_MLP_Train_Task1_Prompt4 --batchsize 256 --modelid 3

python dnn.py CS_10 2 1 CS_MLP_Train_Task2_Prompt1 --batchsize 256 --modelid 3
python dnn.py CS_10 2 2 CS_MLP_Train_Task2_Prompt2 --batchsize 256 --modelid 3
python dnn.py CS_10 2 3 CS_MLP_Train_Task2_Prompt3 --batchsize 256 --modelid 3
python dnn.py CS_10 2 4 CS_MLP_Train_Task2_Prompt4 --batchsize 256 --modelid 3

python dnn.py CS_10 3 1 CS_MLP_Train_Task3_Prompt1 --batchsize 256 --modelid 3
python dnn.py CS_10 3 2 CS_MLP_Train_Task3_Prompt2 --batchsize 256 --modelid 3
python dnn.py CS_10 3 3 CS_MLP_Train_Task3_Prompt3 --batchsize 256 --modelid 3
python dnn.py CS_10 3 4 CS_MLP_Train_Task3_Prompt4 --batchsize 256 --modelid 3

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 1 CS_MLP_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/CS_MLP_Train_Task1_Prompt1/Best_CS_10_Task1.pth
python dnn.py CS_10 1 2 CS_MLP_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/CS_MLP_Train_Task1_Prompt2/Best_CS_10_Task1.pth
python dnn.py CS_10 1 3 CS_MLP_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/CS_MLP_Train_Task1_Prompt3/Best_CS_10_Task1.pth
python dnn.py CS_10 1 4 CS_MLP_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/CS_MLP_Train_Task1_Prompt4/Best_CS_10_Task1.pth

python dnn.py CS_10 2 1 CS_MLP_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/CS_MLP_Train_Task2_Prompt1/Best_CS_10_Task2.pth
python dnn.py CS_10 2 2 CS_MLP_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/CS_MLP_Train_Task2_Prompt2/Best_CS_10_Task2.pth
python dnn.py CS_10 2 3 CS_MLP_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/CS_MLP_Train_Task2_Prompt3/Best_CS_10_Task2.pth
python dnn.py CS_10 2 4 CS_MLP_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/CS_MLP_Train_Task2_Prompt4/Best_CS_10_Task2.pth

python dnn.py CS_10 3 1 CS_MLP_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/CS_MLP_Train_Task3_Prompt1/Best_CS_10_Task3.pth
python dnn.py CS_10 3 2 CS_MLP_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/CS_MLP_Train_Task3_Prompt2/Best_CS_10_Task3.pth
python dnn.py CS_10 3 3 CS_MLP_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/CS_MLP_Train_Task3_Prompt3/Best_CS_10_Task3.pth
python dnn.py CS_10 3 4 CS_MLP_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/CS_MLP_Train_Task3_Prompt4/Best_CS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - AlexNet-like CNN

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 1 CS_CNN_Train_Task1_Prompt1 --batchsize 256 --modelid 4
python dnn.py CS_10 1 2 CS_CNN_Train_Task1_Prompt2 --batchsize 256 --modelid 4
python dnn.py CS_10 1 3 CS_CNN_Train_Task1_Prompt3 --batchsize 256 --modelid 4
python dnn.py CS_10 1 4 CS_CNN_Train_Task1_Prompt4 --batchsize 256 --modelid 4

python dnn.py CS_10 2 1 CS_CNN_Train_Task2_Prompt1 --batchsize 256 --modelid 4
python dnn.py CS_10 2 2 CS_CNN_Train_Task2_Prompt2 --batchsize 256 --modelid 4
python dnn.py CS_10 2 3 CS_CNN_Train_Task2_Prompt3 --batchsize 256 --modelid 4
python dnn.py CS_10 2 4 CS_CNN_Train_Task2_Prompt4 --batchsize 256 --modelid 4

python dnn.py CS_10 3 1 CS_CNN_Train_Task3_Prompt1 --batchsize 256 --modelid 4
python dnn.py CS_10 3 2 CS_CNN_Train_Task3_Prompt2 --batchsize 256 --modelid 4
python dnn.py CS_10 3 3 CS_CNN_Train_Task3_Prompt3 --batchsize 256 --modelid 4
python dnn.py CS_10 3 4 CS_CNN_Train_Task3_Prompt4 --batchsize 256 --modelid 4

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 1 CS_CNN_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/CS_CNN_Train_Task1_Prompt1/Best_CS_10_Task1.pth
python dnn.py CS_10 1 2 CS_CNN_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/CS_CNN_Train_Task1_Prompt2/Best_CS_10_Task1.pth
python dnn.py CS_10 1 3 CS_CNN_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/CS_CNN_Train_Task1_Prompt3/Best_CS_10_Task1.pth
python dnn.py CS_10 1 4 CS_CNN_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/CS_CNN_Train_Task1_Prompt4/Best_CS_10_Task1.pth

python dnn.py CS_10 2 1 CS_CNN_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/CS_CNN_Train_Task2_Prompt1/Best_CS_10_Task2.pth
python dnn.py CS_10 2 2 CS_CNN_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/CS_CNN_Train_Task2_Prompt2/Best_CS_10_Task2.pth
python dnn.py CS_10 2 3 CS_CNN_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/CS_CNN_Train_Task2_Prompt3/Best_CS_10_Task2.pth
python dnn.py CS_10 2 4 CS_CNN_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/CS_CNN_Train_Task2_Prompt4/Best_CS_10_Task2.pth

python dnn.py CS_10 3 1 CS_CNN_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/CS_CNN_Train_Task3_Prompt1/Best_CS_10_Task3.pth
python dnn.py CS_10 3 2 CS_CNN_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/CS_CNN_Train_Task3_Prompt2/Best_CS_10_Task3.pth
python dnn.py CS_10 3 3 CS_CNN_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/CS_CNN_Train_Task3_Prompt3/Best_CS_10_Task3.pth
python dnn.py CS_10 3 4 CS_CNN_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/CS_CNN_Train_Task3_Prompt4/Best_CS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - BiLSTM w/o attention

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 1 CS_BiLSTMwoAttention_Train_Task1_Prompt1 --batchsize 256 --modelid 1
python dnn.py CS_10 1 2 CS_BiLSTMwoAttention_Train_Task1_Prompt2 --batchsize 256 --modelid 1
python dnn.py CS_10 1 3 CS_BiLSTMwoAttention_Train_Task1_Prompt3 --batchsize 256 --modelid 1
python dnn.py CS_10 1 4 CS_BiLSTMwoAttention_Train_Task1_Prompt4 --batchsize 256 --modelid 1

python dnn.py CS_10 2 1 CS_BiLSTMwoAttention_Train_Task2_Prompt1 --batchsize 256 --modelid 1
python dnn.py CS_10 2 2 CS_BiLSTMwoAttention_Train_Task2_Prompt2 --batchsize 256 --modelid 1
python dnn.py CS_10 2 3 CS_BiLSTMwoAttention_Train_Task2_Prompt3 --batchsize 256 --modelid 1
python dnn.py CS_10 2 4 CS_BiLSTMwoAttention_Train_Task2_Prompt4 --batchsize 256 --modelid 1

python dnn.py CS_10 3 1 CS_BiLSTMwoAttention_Train_Task3_Prompt1 --batchsize 256 --modelid 1
python dnn.py CS_10 3 2 CS_BiLSTMwoAttention_Train_Task3_Prompt2 --batchsize 256 --modelid 1
python dnn.py CS_10 3 3 CS_BiLSTMwoAttention_Train_Task3_Prompt3 --batchsize 256 --modelid 1
python dnn.py CS_10 3 4 CS_BiLSTMwoAttention_Train_Task3_Prompt4 --batchsize 256 --modelid 1

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 1 CS_BiLSTMwoAttention_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/CS_BiLSTMwoAttention_Train_Task1_Prompt1/Best_CS_10_Task1.pth
python dnn.py CS_10 1 2 CS_BiLSTMwoAttention_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/CS_BiLSTMwoAttention_Train_Task1_Prompt2/Best_CS_10_Task1.pth
python dnn.py CS_10 1 3 CS_BiLSTMwoAttention_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/CS_BiLSTMwoAttention_Train_Task1_Prompt3/Best_CS_10_Task1.pth
python dnn.py CS_10 1 4 CS_BiLSTMwoAttention_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/CS_BiLSTMwoAttention_Train_Task1_Prompt4/Best_CS_10_Task1.pth

python dnn.py CS_10 2 1 CS_BiLSTMwoAttention_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/CS_BiLSTMwoAttention_Train_Task2_Prompt1/Best_CS_10_Task2.pth
python dnn.py CS_10 2 2 CS_BiLSTMwoAttention_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/CS_BiLSTMwoAttention_Train_Task2_Prompt2/Best_CS_10_Task2.pth
python dnn.py CS_10 2 3 CS_BiLSTMwoAttention_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/CS_BiLSTMwoAttention_Train_Task2_Prompt3/Best_CS_10_Task2.pth
python dnn.py CS_10 2 4 CS_BiLSTMwoAttention_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/CS_BiLSTMwoAttention_Train_Task2_Prompt4/Best_CS_10_Task2.pth

python dnn.py CS_10 3 1 CS_BiLSTMwoAttention_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/CS_BiLSTMwoAttention_Train_Task3_Prompt1/Best_CS_10_Task3.pth
python dnn.py CS_10 3 2 CS_BiLSTMwoAttention_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/CS_BiLSTMwoAttention_Train_Task3_Prompt2/Best_CS_10_Task3.pth
python dnn.py CS_10 3 3 CS_BiLSTMwoAttention_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/CS_BiLSTMwoAttention_Train_Task3_Prompt3/Best_CS_10_Task3.pth
python dnn.py CS_10 3 4 CS_BiLSTMwoAttention_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/CS_BiLSTMwoAttention_Train_Task3_Prompt4/Best_CS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 9 - Cross Prompt

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 0 CS_CheckGPTArch_Train_Task1_Prompt1234_Test_Task1_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/CS_Task1_Prompt_1234/Best_CS_Task1.pth
python dnn.py CS_10 2 0 CS_CheckGPTArch_Train_Task2_Prompt1234_Test_Task2_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/CS_Task2_Prompt1234/Best_CS_Task2.pth
python dnn.py CS_10 3 0 CS_CheckGPTArch_Train_Task3_Prompt1234_Test_Task3_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/CS_Task3_Prompt1234/Best_CS_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 234

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 234 CS_CheckGPTArch_Train_Task1_Prompt234 --batchsize 256
python dnn.py CS_10 2 234 CS_CheckGPTArch_Train_Task2_Prompt234 --batchsize 256
python dnn.py CS_10 3 234 CS_CheckGPTArch_Train_Task3_Prompt234 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 1 CS_CheckGPTArch_Train_Task1_Prompt234_Test_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task1_Prompt234/Best_CS_10_Task1.pth
python dnn.py CS_10 2 1 CS_CheckGPTArch_Train_Task2_Prompt234_Test_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task2_Prompt234/Best_CS_10_Task2.pth
python dnn.py CS_10 3 1 CS_CheckGPTArch_Train_Task3_Prompt234_Test_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task3_Prompt234/Best_CS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 134

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 134 CS_CheckGPTArch_Train_Task1_Prompt134 --batchsize 256
python dnn.py CS_10 2 134 CS_CheckGPTArch_Train_Task2_Prompt134 --batchsize 256
python dnn.py CS_10 3 134 CS_CheckGPTArch_Train_Task3_Prompt134 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 2 CS_CheckGPTArch_Train_Task1_Prompt134_Test_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task1_Prompt134/Best_CS_10_Task1.pth
python dnn.py CS_10 2 2 CS_CheckGPTArch_Train_Task2_Prompt134_Test_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task2_Prompt134/Best_CS_10_Task2.pth
python dnn.py CS_10 3 2 CS_CheckGPTArch_Train_Task3_Prompt134_Test_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task3_Prompt134/Best_CS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 124

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 124 CS_CheckGPTArch_Train_Task1_Prompt124 --batchsize 256
python dnn.py CS_10 2 124 CS_CheckGPTArch_Train_Task2_Prompt124 --batchsize 256
python dnn.py CS_10 3 124 CS_CheckGPTArch_Train_Task3_Prompt124 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 3 CS_CheckGPTArch_Train_Task1_Prompt124_Test_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task1_Prompt124/Best_CS_10_Task1.pth
python dnn.py CS_10 2 3 CS_CheckGPTArch_Train_Task2_Prompt124_Test_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task2_Prompt124/Best_CS_10_Task2.pth
python dnn.py CS_10 3 3 CS_CheckGPTArch_Train_Task3_Prompt124_Test_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task3_Prompt124/Best_CS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 123

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 123 CS_CheckGPTArch_Train_Task1_Prompt123 --batchsize 256
python dnn.py CS_10 2 123 CS_CheckGPTArch_Train_Task2_Prompt123 --batchsize 256
python dnn.py CS_10 3 123 CS_CheckGPTArch_Train_Task3_Prompt123 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_10 1 4 CS_CheckGPTArch_Train_Task1_Prompt123_Test_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task1_Prompt123/Best_CS_10_Task1.pth
python dnn.py CS_10 2 4 CS_CheckGPTArch_Train_Task2_Prompt123_Test_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task2_Prompt123/Best_CS_10_Task2.pth
python dnn.py CS_10 3 4 CS_CheckGPTArch_Train_Task3_Prompt123_Test_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/CS_CheckGPTArch_Train_Task3_Prompt123/Best_CS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Feature Clean-up

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/
shopt -s extglob
rm -rf embeddings/!(README.md)

## PHX - 10

In [ ]:
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python features.py PHX 1 1 --gpt 0 --number 10
python features.py PHX 2 1 --gpt 0 --number 10

python features.py PHX 1 1 --gpt 1 --number 10
python features.py PHX 1 2 --gpt 1 --number 10
python features.py PHX 1 3 --gpt 1 --number 10
python features.py PHX 1 4 --gpt 1 --number 10

python features.py PHX 2 1 --gpt 1 --number 10
python features.py PHX 2 2 --gpt 1 --number 10
python features.py PHX 2 3 --gpt 1 --number 10
python features.py PHX 2 4 --gpt 1 --number 10

python features.py PHX 3 1 --gpt 1 --number 10
python features.py PHX 3 2 --gpt 1 --number 10
python features.py PHX 3 3 --gpt 1 --number 10
python features.py PHX 3 4 --gpt 1 --number 10

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - CheckGPT

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 1 PHX_CheckGPTArch_Train_Task1_Prompt1 --batchsize 256
python dnn.py PHX_10 1 2 PHX_CheckGPTArch_Train_Task1_Prompt2 --batchsize 256
python dnn.py PHX_10 1 3 PHX_CheckGPTArch_Train_Task1_Prompt3 --batchsize 256
python dnn.py PHX_10 1 4 PHX_CheckGPTArch_Train_Task1_Prompt4 --batchsize 256

python dnn.py PHX_10 2 1 PHX_CheckGPTArch_Train_Task2_Prompt1 --batchsize 256
python dnn.py PHX_10 2 2 PHX_CheckGPTArch_Train_Task2_Prompt2 --batchsize 256
python dnn.py PHX_10 2 3 PHX_CheckGPTArch_Train_Task2_Prompt3 --batchsize 256
python dnn.py PHX_10 2 4 PHX_CheckGPTArch_Train_Task2_Prompt4 --batchsize 256

python dnn.py PHX_10 3 1 PHX_CheckGPTArch_Train_Task3_Prompt1 --batchsize 256
python dnn.py PHX_10 3 2 PHX_CheckGPTArch_Train_Task3_Prompt2 --batchsize 256
python dnn.py PHX_10 3 3 PHX_CheckGPTArch_Train_Task3_Prompt3 --batchsize 256
python dnn.py PHX_10 3 4 PHX_CheckGPTArch_Train_Task3_Prompt4 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 1 PHX_CheckGPTArch_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task1_Prompt1/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 2 PHX_CheckGPTArch_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task1_Prompt2/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 3 PHX_CheckGPTArch_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task1_Prompt3/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 4 PHX_CheckGPTArch_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task1_Prompt4/Best_PHX_10_Task1.pth

python dnn.py PHX_10 2 1 PHX_CheckGPTArch_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task2_Prompt1/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 2 PHX_CheckGPTArch_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task2_Prompt2/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 3 PHX_CheckGPTArch_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task2_Prompt3/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 4 PHX_CheckGPTArch_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task2_Prompt4/Best_PHX_10_Task2.pth

python dnn.py PHX_10 3 1 PHX_CheckGPTArch_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task3_Prompt1/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 2 PHX_CheckGPTArch_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task3_Prompt2/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 3 PHX_CheckGPTArch_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task3_Prompt3/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 4 PHX_CheckGPTArch_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task3_Prompt4/Best_PHX_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - RCH

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 1 PHX_RCH_Train_Task1_Prompt1 --batchsize 256 --modelid 2
python dnn.py PHX_10 1 2 PHX_RCH_Train_Task1_Prompt2 --batchsize 256 --modelid 2
python dnn.py PHX_10 1 3 PHX_RCH_Train_Task1_Prompt3 --batchsize 256 --modelid 2
python dnn.py PHX_10 1 4 PHX_RCH_Train_Task1_Prompt4 --batchsize 256 --modelid 2

python dnn.py PHX_10 2 1 PHX_RCH_Train_Task2_Prompt1 --batchsize 256 --modelid 2
python dnn.py PHX_10 2 2 PHX_RCH_Train_Task2_Prompt2 --batchsize 256 --modelid 2
python dnn.py PHX_10 2 3 PHX_RCH_Train_Task2_Prompt3 --batchsize 256 --modelid 2
python dnn.py PHX_10 2 4 PHX_RCH_Train_Task2_Prompt4 --batchsize 256 --modelid 2

python dnn.py PHX_10 3 1 PHX_RCH_Train_Task3_Prompt1 --batchsize 256 --modelid 2
python dnn.py PHX_10 3 2 PHX_RCH_Train_Task3_Prompt2 --batchsize 256 --modelid 2
python dnn.py PHX_10 3 3 PHX_RCH_Train_Task3_Prompt3 --batchsize 256 --modelid 2
python dnn.py PHX_10 3 4 PHX_RCH_Train_Task3_Prompt4 --batchsize 256 --modelid 2

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 1 PHX_RCH_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/PHX_RCH_Train_Task1_Prompt1/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 2 PHX_RCH_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/PHX_RCH_Train_Task1_Prompt2/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 3 PHX_RCH_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/PHX_RCH_Train_Task1_Prompt3/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 4 PHX_RCH_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/PHX_RCH_Train_Task1_Prompt4/Best_PHX_10_Task1.pth

python dnn.py PHX_10 2 1 PHX_RCH_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/PHX_RCH_Train_Task2_Prompt1/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 2 PHX_RCH_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/PHX_RCH_Train_Task2_Prompt2/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 3 PHX_RCH_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/PHX_RCH_Train_Task2_Prompt3/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 4 PHX_RCH_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/PHX_RCH_Train_Task2_Prompt4/Best_PHX_10_Task2.pth

python dnn.py PHX_10 3 1 PHX_RCH_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/PHX_RCH_Train_Task3_Prompt1/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 2 PHX_RCH_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/PHX_RCH_Train_Task3_Prompt2/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 3 PHX_RCH_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/PHX_RCH_Train_Task3_Prompt3/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 4 PHX_RCH_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/PHX_RCH_Train_Task3_Prompt4/Best_PHX_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - MLP-Pool

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 1 PHX_MLP_Train_Task1_Prompt1 --batchsize 256 --modelid 3
python dnn.py PHX_10 1 2 PHX_MLP_Train_Task1_Prompt2 --batchsize 256 --modelid 3
python dnn.py PHX_10 1 3 PHX_MLP_Train_Task1_Prompt3 --batchsize 256 --modelid 3
python dnn.py PHX_10 1 4 PHX_MLP_Train_Task1_Prompt4 --batchsize 256 --modelid 3

python dnn.py PHX_10 2 1 PHX_MLP_Train_Task2_Prompt1 --batchsize 256 --modelid 3
python dnn.py PHX_10 2 2 PHX_MLP_Train_Task2_Prompt2 --batchsize 256 --modelid 3
python dnn.py PHX_10 2 3 PHX_MLP_Train_Task2_Prompt3 --batchsize 256 --modelid 3
python dnn.py PHX_10 2 4 PHX_MLP_Train_Task2_Prompt4 --batchsize 256 --modelid 3

python dnn.py PHX_10 3 1 PHX_MLP_Train_Task3_Prompt1 --batchsize 256 --modelid 3
python dnn.py PHX_10 3 2 PHX_MLP_Train_Task3_Prompt2 --batchsize 256 --modelid 3
python dnn.py PHX_10 3 3 PHX_MLP_Train_Task3_Prompt3 --batchsize 256 --modelid 3
python dnn.py PHX_10 3 4 PHX_MLP_Train_Task3_Prompt4 --batchsize 256 --modelid 3

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 1 PHX_MLP_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/PHX_MLP_Train_Task1_Prompt1/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 2 PHX_MLP_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/PHX_MLP_Train_Task1_Prompt2/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 3 PHX_MLP_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/PHX_MLP_Train_Task1_Prompt3/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 4 PHX_MLP_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/PHX_MLP_Train_Task1_Prompt4/Best_PHX_10_Task1.pth

python dnn.py PHX_10 2 1 PHX_MLP_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/PHX_MLP_Train_Task2_Prompt1/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 2 PHX_MLP_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/PHX_MLP_Train_Task2_Prompt2/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 3 PHX_MLP_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/PHX_MLP_Train_Task2_Prompt3/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 4 PHX_MLP_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/PHX_MLP_Train_Task2_Prompt4/Best_PHX_10_Task2.pth

python dnn.py PHX_10 3 1 PHX_MLP_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/PHX_MLP_Train_Task3_Prompt1/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 2 PHX_MLP_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/PHX_MLP_Train_Task3_Prompt2/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 3 PHX_MLP_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/PHX_MLP_Train_Task3_Prompt3/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 4 PHX_MLP_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/PHX_MLP_Train_Task3_Prompt4/Best_PHX_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - AlexNet-like CNN

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 1 PHX_CNN_Train_Task1_Prompt1 --batchsize 256 --modelid 4
python dnn.py PHX_10 1 2 PHX_CNN_Train_Task1_Prompt2 --batchsize 256 --modelid 4
python dnn.py PHX_10 1 3 PHX_CNN_Train_Task1_Prompt3 --batchsize 256 --modelid 4
python dnn.py PHX_10 1 4 PHX_CNN_Train_Task1_Prompt4 --batchsize 256 --modelid 4

python dnn.py PHX_10 2 1 PHX_CNN_Train_Task2_Prompt1 --batchsize 256 --modelid 4
python dnn.py PHX_10 2 2 PHX_CNN_Train_Task2_Prompt2 --batchsize 256 --modelid 4
python dnn.py PHX_10 2 3 PHX_CNN_Train_Task2_Prompt3 --batchsize 256 --modelid 4
python dnn.py PHX_10 2 4 PHX_CNN_Train_Task2_Prompt4 --batchsize 256 --modelid 4

python dnn.py PHX_10 3 1 PHX_CNN_Train_Task3_Prompt1 --batchsize 256 --modelid 4
python dnn.py PHX_10 3 2 PHX_CNN_Train_Task3_Prompt2 --batchsize 256 --modelid 4
python dnn.py PHX_10 3 3 PHX_CNN_Train_Task3_Prompt3 --batchsize 256 --modelid 4
python dnn.py PHX_10 3 4 PHX_CNN_Train_Task3_Prompt4 --batchsize 256 --modelid 4

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 1 PHX_CNN_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/PHX_CNN_Train_Task1_Prompt1/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 2 PHX_CNN_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/PHX_CNN_Train_Task1_Prompt2/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 3 PHX_CNN_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/PHX_CNN_Train_Task1_Prompt3/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 4 PHX_CNN_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/PHX_CNN_Train_Task1_Prompt4/Best_PHX_10_Task1.pth

python dnn.py PHX_10 2 1 PHX_CNN_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/PHX_CNN_Train_Task2_Prompt1/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 2 PHX_CNN_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/PHX_CNN_Train_Task2_Prompt2/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 3 PHX_CNN_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/PHX_CNN_Train_Task2_Prompt3/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 4 PHX_CNN_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/PHX_CNN_Train_Task2_Prompt4/Best_PHX_10_Task2.pth

python dnn.py PHX_10 3 1 PHX_CNN_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/PHX_CNN_Train_Task3_Prompt1/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 2 PHX_CNN_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/PHX_CNN_Train_Task3_Prompt2/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 3 PHX_CNN_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/PHX_CNN_Train_Task3_Prompt3/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 4 PHX_CNN_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/PHX_CNN_Train_Task3_Prompt4/Best_PHX_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - BiLSTM w/o attention

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 1 PHX_BiLSTMwoAttention_Train_Task1_Prompt1 --batchsize 256 --modelid 1
python dnn.py PHX_10 1 2 PHX_BiLSTMwoAttention_Train_Task1_Prompt2 --batchsize 256 --modelid 1
python dnn.py PHX_10 1 3 PHX_BiLSTMwoAttention_Train_Task1_Prompt3 --batchsize 256 --modelid 1
python dnn.py PHX_10 1 4 PHX_BiLSTMwoAttention_Train_Task1_Prompt4 --batchsize 256 --modelid 1

python dnn.py PHX_10 2 1 PHX_BiLSTMwoAttention_Train_Task2_Prompt1 --batchsize 256 --modelid 1
python dnn.py PHX_10 2 2 PHX_BiLSTMwoAttention_Train_Task2_Prompt2 --batchsize 256 --modelid 1
python dnn.py PHX_10 2 3 PHX_BiLSTMwoAttention_Train_Task2_Prompt3 --batchsize 256 --modelid 1
python dnn.py PHX_10 2 4 PHX_BiLSTMwoAttention_Train_Task2_Prompt4 --batchsize 256 --modelid 1

python dnn.py PHX_10 3 1 PHX_BiLSTMwoAttention_Train_Task3_Prompt1 --batchsize 256 --modelid 1
python dnn.py PHX_10 3 2 PHX_BiLSTMwoAttention_Train_Task3_Prompt2 --batchsize 256 --modelid 1
python dnn.py PHX_10 3 3 PHX_BiLSTMwoAttention_Train_Task3_Prompt3 --batchsize 256 --modelid 1
python dnn.py PHX_10 3 4 PHX_BiLSTMwoAttention_Train_Task3_Prompt4 --batchsize 256 --modelid 1

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 1 PHX_BiLSTMwoAttention_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/PHX_BiLSTMwoAttention_Train_Task1_Prompt1/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 2 PHX_BiLSTMwoAttention_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/PHX_BiLSTMwoAttention_Train_Task1_Prompt2/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 3 PHX_BiLSTMwoAttention_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/PHX_BiLSTMwoAttention_Train_Task1_Prompt3/Best_PHX_10_Task1.pth
python dnn.py PHX_10 1 4 PHX_BiLSTMwoAttention_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/PHX_BiLSTMwoAttention_Train_Task1_Prompt4/Best_PHX_10_Task1.pth

python dnn.py PHX_10 2 1 PHX_BiLSTMwoAttention_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/PHX_BiLSTMwoAttention_Train_Task2_Prompt1/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 2 PHX_BiLSTMwoAttention_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/PHX_BiLSTMwoAttention_Train_Task2_Prompt2/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 3 PHX_BiLSTMwoAttention_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/PHX_BiLSTMwoAttention_Train_Task2_Prompt3/Best_PHX_10_Task2.pth
python dnn.py PHX_10 2 4 PHX_BiLSTMwoAttention_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/PHX_BiLSTMwoAttention_Train_Task2_Prompt4/Best_PHX_10_Task2.pth

python dnn.py PHX_10 3 1 PHX_BiLSTMwoAttention_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/PHX_BiLSTMwoAttention_Train_Task3_Prompt1/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 2 PHX_BiLSTMwoAttention_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/PHX_BiLSTMwoAttention_Train_Task3_Prompt2/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 3 PHX_BiLSTMwoAttention_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/PHX_BiLSTMwoAttention_Train_Task3_Prompt3/Best_PHX_10_Task3.pth
python dnn.py PHX_10 3 4 PHX_BiLSTMwoAttention_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/PHX_BiLSTMwoAttention_Train_Task3_Prompt4/Best_PHX_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 9 - Cross Prompt

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 0 PHX_CheckGPTArch_Train_Task1_Prompt1234_Test_Task1_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/PHX_Task1_Prompt1234/Best_PHY_Task1.pth
python dnn.py PHX_10 2 0 PHX_CheckGPTArch_Train_Task2_Prompt1234_Test_Task2_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/PHX_Task2_Prompt1234/Best_PHY_Task2.pth
python dnn.py PHX_10 3 0 PHX_CheckGPTArch_Train_Task3_Prompt1234_Test_Task3_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/PHX_Task3_Prompt1234/Best_PHY_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 234

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 234 PHX_CheckGPTArch_Train_Task1_Prompt234 --batchsize 256
python dnn.py PHX_10 2 234 PHX_CheckGPTArch_Train_Task2_Prompt234 --batchsize 256
python dnn.py PHX_10 3 234 PHX_CheckGPTArch_Train_Task3_Prompt234 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 1 PHX_CheckGPTArch_Train_Task1_Prompt234_Test_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task1_Prompt234/Best_PHX_10_Task1.pth
python dnn.py PHX_10 2 1 PHX_CheckGPTArch_Train_Task2_Prompt234_Test_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task2_Prompt234/Best_PHX_10_Task2.pth
python dnn.py PHX_10 3 1 PHX_CheckGPTArch_Train_Task3_Prompt234_Test_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task3_Prompt234/Best_PHX_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 134

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 134 PHX_CheckGPTArch_Train_Task1_Prompt134 --batchsize 256
python dnn.py PHX_10 2 134 PHX_CheckGPTArch_Train_Task2_Prompt134 --batchsize 256
python dnn.py PHX_10 3 134 PHX_CheckGPTArch_Train_Task3_Prompt134 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 2 PHX_CheckGPTArch_Train_Task1_Prompt134_Test_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task1_Prompt134/Best_PHX_10_Task1.pth
python dnn.py PHX_10 2 2 PHX_CheckGPTArch_Train_Task2_Prompt134_Test_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task2_Prompt134/Best_PHX_10_Task2.pth
python dnn.py PHX_10 3 2 PHX_CheckGPTArch_Train_Task3_Prompt134_Test_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task3_Prompt134/Best_PHX_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 124

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 124 PHX_CheckGPTArch_Train_Task1_Prompt124 --batchsize 256
python dnn.py PHX_10 2 124 PHX_CheckGPTArch_Train_Task2_Prompt124 --batchsize 256
python dnn.py PHX_10 3 124 PHX_CheckGPTArch_Train_Task3_Prompt124 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 3 PHX_CheckGPTArch_Train_Task1_Prompt124_Test_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task1_Prompt124/Best_PHX_10_Task1.pth
python dnn.py PHX_10 2 3 PHX_CheckGPTArch_Train_Task2_Prompt124_Test_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task2_Prompt124/Best_PHX_10_Task2.pth
python dnn.py PHX_10 3 3 PHX_CheckGPTArch_Train_Task3_Prompt124_Test_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task3_Prompt124/Best_PHX_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 123

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 123 PHX_CheckGPTArch_Train_Task1_Prompt123 --batchsize 256
python dnn.py PHX_10 2 123 PHX_CheckGPTArch_Train_Task2_Prompt123 --batchsize 256
python dnn.py PHX_10 3 123 PHX_CheckGPTArch_Train_Task3_Prompt123 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_10 1 4 PHX_CheckGPTArch_Train_Task1_Prompt123_Test_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task1_Prompt123/Best_PHX_10_Task1.pth
python dnn.py PHX_10 2 4 PHX_CheckGPTArch_Train_Task2_Prompt123_Test_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task2_Prompt123/Best_PHX_10_Task2.pth
python dnn.py PHX_10 3 4 PHX_CheckGPTArch_Train_Task3_Prompt123_Test_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/PHX_CheckGPTArch_Train_Task3_Prompt123/Best_PHX_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Feature Clean-up

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/
shopt -s extglob
rm -rf embeddings/!(README.md)

## HSS - 10

In [ ]:
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python features.py HSS 1 1 --gpt 0 --number 10
python features.py HSS 2 1 --gpt 0 --number 10

python features.py HSS 1 1 --gpt 1 --number 10
python features.py HSS 1 2 --gpt 1 --number 10
python features.py HSS 1 3 --gpt 1 --number 10
python features.py HSS 1 4 --gpt 1 --number 10

python features.py HSS 2 1 --gpt 1 --number 10
python features.py HSS 2 2 --gpt 1 --number 10
python features.py HSS 2 3 --gpt 1 --number 10
python features.py HSS 2 4 --gpt 1 --number 10

python features.py HSS 3 1 --gpt 1 --number 10
python features.py HSS 3 2 --gpt 1 --number 10
python features.py HSS 3 3 --gpt 1 --number 10
python features.py HSS 3 4 --gpt 1 --number 10

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - HSS - CheckGPT

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 1 HSS_CheckGPTArch_Train_Task1_Prompt1 --batchsize 256
python dnn.py HSS_10 1 2 HSS_CheckGPTArch_Train_Task1_Prompt2 --batchsize 256
python dnn.py HSS_10 1 3 HSS_CheckGPTArch_Train_Task1_Prompt3 --batchsize 256
python dnn.py HSS_10 1 4 HSS_CheckGPTArch_Train_Task1_Prompt4 --batchsize 256

python dnn.py HSS_10 2 1 HSS_CheckGPTArch_Train_Task2_Prompt1 --batchsize 256
python dnn.py HSS_10 2 2 HSS_CheckGPTArch_Train_Task2_Prompt2 --batchsize 256
python dnn.py HSS_10 2 3 HSS_CheckGPTArch_Train_Task2_Prompt3 --batchsize 256
python dnn.py HSS_10 2 4 HSS_CheckGPTArch_Train_Task2_Prompt4 --batchsize 256

python dnn.py HSS_10 3 1 HSS_CheckGPTArch_Train_Task3_Prompt1 --batchsize 256
python dnn.py HSS_10 3 2 HSS_CheckGPTArch_Train_Task3_Prompt2 --batchsize 256
python dnn.py HSS_10 3 3 HSS_CheckGPTArch_Train_Task3_Prompt3 --batchsize 256
python dnn.py HSS_10 3 4 HSS_CheckGPTArch_Train_Task3_Prompt4 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 1 HSS_CheckGPTArch_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task1_Prompt1/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 2 HSS_CheckGPTArch_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task1_Prompt2/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 3 HSS_CheckGPTArch_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task1_Prompt3/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 4 HSS_CheckGPTArch_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task1_Prompt4/Best_HSS_10_Task1.pth

python dnn.py HSS_10 2 1 HSS_CheckGPTArch_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task2_Prompt1/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 2 HSS_CheckGPTArch_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task2_Prompt2/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 3 HSS_CheckGPTArch_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task2_Prompt3/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 4 HSS_CheckGPTArch_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task2_Prompt4/Best_HSS_10_Task2.pth

python dnn.py HSS_10 3 1 HSS_CheckGPTArch_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task3_Prompt1/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 2 HSS_CheckGPTArch_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task3_Prompt2/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 3 HSS_CheckGPTArch_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task3_Prompt3/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 4 HSS_CheckGPTArch_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task3_Prompt4/Best_HSS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - HSS - RCH

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 1 HSS_RCH_Train_Task1_Prompt1 --batchsize 256 --modelid 2
python dnn.py HSS_10 1 2 HSS_RCH_Train_Task1_Prompt2 --batchsize 256 --modelid 2
python dnn.py HSS_10 1 3 HSS_RCH_Train_Task1_Prompt3 --batchsize 256 --modelid 2
python dnn.py HSS_10 1 4 HSS_RCH_Train_Task1_Prompt4 --batchsize 256 --modelid 2

python dnn.py HSS_10 2 1 HSS_RCH_Train_Task2_Prompt1 --batchsize 256 --modelid 2
python dnn.py HSS_10 2 2 HSS_RCH_Train_Task2_Prompt2 --batchsize 256 --modelid 2
python dnn.py HSS_10 2 3 HSS_RCH_Train_Task2_Prompt3 --batchsize 256 --modelid 2
python dnn.py HSS_10 2 4 HSS_RCH_Train_Task2_Prompt4 --batchsize 256 --modelid 2

python dnn.py HSS_10 3 1 HSS_RCH_Train_Task3_Prompt1 --batchsize 256 --modelid 2
python dnn.py HSS_10 3 2 HSS_RCH_Train_Task3_Prompt2 --batchsize 256 --modelid 2
python dnn.py HSS_10 3 3 HSS_RCH_Train_Task3_Prompt3 --batchsize 256 --modelid 2
python dnn.py HSS_10 3 4 HSS_RCH_Train_Task3_Prompt4 --batchsize 256 --modelid 2

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 1 HSS_RCH_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/HSS_RCH_Train_Task1_Prompt1/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 2 HSS_RCH_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/HSS_RCH_Train_Task1_Prompt2/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 3 HSS_RCH_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/HSS_RCH_Train_Task1_Prompt3/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 4 HSS_RCH_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/HSS_RCH_Train_Task1_Prompt4/Best_HSS_10_Task1.pth

python dnn.py HSS_10 2 1 HSS_RCH_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/HSS_RCH_Train_Task2_Prompt1/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 2 HSS_RCH_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/HSS_RCH_Train_Task2_Prompt2/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 3 HSS_RCH_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/HSS_RCH_Train_Task2_Prompt3/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 4 HSS_RCH_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/HSS_RCH_Train_Task2_Prompt4/Best_HSS_10_Task2.pth

python dnn.py HSS_10 3 1 HSS_RCH_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/HSS_RCH_Train_Task3_Prompt1/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 2 HSS_RCH_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/HSS_RCH_Train_Task3_Prompt2/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 3 HSS_RCH_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/HSS_RCH_Train_Task3_Prompt3/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 4 HSS_RCH_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --modelid 2 --saved-model ./exp/HSS_RCH_Train_Task3_Prompt4/Best_HSS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - HSS - MLP-Pool

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 1 HSS_MLP_Train_Task1_Prompt1 --batchsize 256 --modelid 3
python dnn.py HSS_10 1 2 HSS_MLP_Train_Task1_Prompt2 --batchsize 256 --modelid 3
python dnn.py HSS_10 1 3 HSS_MLP_Train_Task1_Prompt3 --batchsize 256 --modelid 3
python dnn.py HSS_10 1 4 HSS_MLP_Train_Task1_Prompt4 --batchsize 256 --modelid 3

python dnn.py HSS_10 2 1 HSS_MLP_Train_Task2_Prompt1 --batchsize 256 --modelid 3
python dnn.py HSS_10 2 2 HSS_MLP_Train_Task2_Prompt2 --batchsize 256 --modelid 3
python dnn.py HSS_10 2 3 HSS_MLP_Train_Task2_Prompt3 --batchsize 256 --modelid 3
python dnn.py HSS_10 2 4 HSS_MLP_Train_Task2_Prompt4 --batchsize 256 --modelid 3

python dnn.py HSS_10 3 1 HSS_MLP_Train_Task3_Prompt1 --batchsize 256 --modelid 3
python dnn.py HSS_10 3 2 HSS_MLP_Train_Task3_Prompt2 --batchsize 256 --modelid 3
python dnn.py HSS_10 3 3 HSS_MLP_Train_Task3_Prompt3 --batchsize 256 --modelid 3
python dnn.py HSS_10 3 4 HSS_MLP_Train_Task3_Prompt4 --batchsize 256 --modelid 3

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 1 HSS_MLP_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/HSS_MLP_Train_Task1_Prompt1/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 2 HSS_MLP_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/HSS_MLP_Train_Task1_Prompt2/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 3 HSS_MLP_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/HSS_MLP_Train_Task1_Prompt3/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 4 HSS_MLP_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/HSS_MLP_Train_Task1_Prompt4/Best_HSS_10_Task1.pth

python dnn.py HSS_10 2 1 HSS_MLP_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/HSS_MLP_Train_Task2_Prompt1/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 2 HSS_MLP_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/HSS_MLP_Train_Task2_Prompt2/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 3 HSS_MLP_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/HSS_MLP_Train_Task2_Prompt3/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 4 HSS_MLP_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/HSS_MLP_Train_Task2_Prompt4/Best_HSS_10_Task2.pth

python dnn.py HSS_10 3 1 HSS_MLP_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/HSS_MLP_Train_Task3_Prompt1/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 2 HSS_MLP_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/HSS_MLP_Train_Task3_Prompt2/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 3 HSS_MLP_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/HSS_MLP_Train_Task3_Prompt3/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 4 HSS_MLP_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --modelid 3 --saved-model ./exp/HSS_MLP_Train_Task3_Prompt4/Best_HSS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - HSS - AlexNet-like CNN

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 1 HSS_CNN_Train_Task1_Prompt1 --batchsize 256 --modelid 4
python dnn.py HSS_10 1 2 HSS_CNN_Train_Task1_Prompt2 --batchsize 256 --modelid 4
python dnn.py HSS_10 1 3 HSS_CNN_Train_Task1_Prompt3 --batchsize 256 --modelid 4
python dnn.py HSS_10 1 4 HSS_CNN_Train_Task1_Prompt4 --batchsize 256 --modelid 4

python dnn.py HSS_10 2 1 HSS_CNN_Train_Task2_Prompt1 --batchsize 256 --modelid 4
python dnn.py HSS_10 2 2 HSS_CNN_Train_Task2_Prompt2 --batchsize 256 --modelid 4
python dnn.py HSS_10 2 3 HSS_CNN_Train_Task2_Prompt3 --batchsize 256 --modelid 4
python dnn.py HSS_10 2 4 HSS_CNN_Train_Task2_Prompt4 --batchsize 256 --modelid 4

python dnn.py HSS_10 3 1 HSS_CNN_Train_Task3_Prompt1 --batchsize 256 --modelid 4
python dnn.py HSS_10 3 2 HSS_CNN_Train_Task3_Prompt2 --batchsize 256 --modelid 4
python dnn.py HSS_10 3 3 HSS_CNN_Train_Task3_Prompt3 --batchsize 256 --modelid 4
python dnn.py HSS_10 3 4 HSS_CNN_Train_Task3_Prompt4 --batchsize 256 --modelid 4

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 1 HSS_CNN_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/HSS_CNN_Train_Task1_Prompt1/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 2 HSS_CNN_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/HSS_CNN_Train_Task1_Prompt2/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 3 HSS_CNN_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/HSS_CNN_Train_Task1_Prompt3/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 4 HSS_CNN_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/HSS_CNN_Train_Task1_Prompt4/Best_HSS_10_Task1.pth

python dnn.py HSS_10 2 1 HSS_CNN_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/HSS_CNN_Train_Task2_Prompt1/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 2 HSS_CNN_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/HSS_CNN_Train_Task2_Prompt2/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 3 HSS_CNN_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/HSS_CNN_Train_Task2_Prompt3/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 4 HSS_CNN_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/HSS_CNN_Train_Task2_Prompt4/Best_HSS_10_Task2.pth

python dnn.py HSS_10 3 1 HSS_CNN_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/HSS_CNN_Train_Task3_Prompt1/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 2 HSS_CNN_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/HSS_CNN_Train_Task3_Prompt2/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 3 HSS_CNN_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/HSS_CNN_Train_Task3_Prompt3/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 4 HSS_CNN_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --modelid 4 --saved-model ./exp/HSS_CNN_Train_Task3_Prompt4/Best_HSS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 7 - HSS - BiLSTM w/o attention

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 1 HSS_BiLSTMwoAttention_Train_Task1_Prompt1 --batchsize 256 --modelid 1
python dnn.py HSS_10 1 2 HSS_BiLSTMwoAttention_Train_Task1_Prompt2 --batchsize 256 --modelid 1
python dnn.py HSS_10 1 3 HSS_BiLSTMwoAttention_Train_Task1_Prompt3 --batchsize 256 --modelid 1
python dnn.py HSS_10 1 4 HSS_BiLSTMwoAttention_Train_Task1_Prompt4 --batchsize 256 --modelid 1

python dnn.py HSS_10 2 1 HSS_BiLSTMwoAttention_Train_Task2_Prompt1 --batchsize 256 --modelid 1
python dnn.py HSS_10 2 2 HSS_BiLSTMwoAttention_Train_Task2_Prompt2 --batchsize 256 --modelid 1
python dnn.py HSS_10 2 3 HSS_BiLSTMwoAttention_Train_Task2_Prompt3 --batchsize 256 --modelid 1
python dnn.py HSS_10 2 4 HSS_BiLSTMwoAttention_Train_Task2_Prompt4 --batchsize 256 --modelid 1

python dnn.py HSS_10 3 1 HSS_BiLSTMwoAttention_Train_Task3_Prompt1 --batchsize 256 --modelid 1
python dnn.py HSS_10 3 2 HSS_BiLSTMwoAttention_Train_Task3_Prompt2 --batchsize 256 --modelid 1
python dnn.py HSS_10 3 3 HSS_BiLSTMwoAttention_Train_Task3_Prompt3 --batchsize 256 --modelid 1
python dnn.py HSS_10 3 4 HSS_BiLSTMwoAttention_Train_Task3_Prompt4 --batchsize 256 --modelid 1

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 1 HSS_BiLSTMwoAttention_Train_Task1_Prompt1_Test_Task1_Prompt1 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/HSS_BiLSTMwoAttention_Train_Task1_Prompt1/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 2 HSS_BiLSTMwoAttention_Train_Task1_Prompt2_Test_Task1_Prompt2 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/HSS_BiLSTMwoAttention_Train_Task1_Prompt2/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 3 HSS_BiLSTMwoAttention_Train_Task1_Prompt3_Test_Task1_Prompt3 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/HSS_BiLSTMwoAttention_Train_Task1_Prompt3/Best_HSS_10_Task1.pth
python dnn.py HSS_10 1 4 HSS_BiLSTMwoAttention_Train_Task1_Prompt4_Test_Task1_Prompt4 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/HSS_BiLSTMwoAttention_Train_Task1_Prompt4/Best_HSS_10_Task1.pth

python dnn.py HSS_10 2 1 HSS_BiLSTMwoAttention_Train_Task2_Prompt1_Test_Task2_Prompt1 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/HSS_BiLSTMwoAttention_Train_Task2_Prompt1/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 2 HSS_BiLSTMwoAttention_Train_Task2_Prompt2_Test_Task2_Prompt2 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/HSS_BiLSTMwoAttention_Train_Task2_Prompt2/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 3 HSS_BiLSTMwoAttention_Train_Task2_Prompt3_Test_Task2_Prompt3 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/HSS_BiLSTMwoAttention_Train_Task2_Prompt3/Best_HSS_10_Task2.pth
python dnn.py HSS_10 2 4 HSS_BiLSTMwoAttention_Train_Task2_Prompt4_Test_Task2_Prompt4 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/HSS_BiLSTMwoAttention_Train_Task2_Prompt4/Best_HSS_10_Task2.pth

python dnn.py HSS_10 3 1 HSS_BiLSTMwoAttention_Train_Task3_Prompt1_Test_Task3_Prompt1 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/HSS_BiLSTMwoAttention_Train_Task3_Prompt1/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 2 HSS_BiLSTMwoAttention_Train_Task3_Prompt2_Test_Task3_Prompt2 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/HSS_BiLSTMwoAttention_Train_Task3_Prompt2/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 3 HSS_BiLSTMwoAttention_Train_Task3_Prompt3_Test_Task3_Prompt3 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/HSS_BiLSTMwoAttention_Train_Task3_Prompt3/Best_HSS_10_Task3.pth
python dnn.py HSS_10 3 4 HSS_BiLSTMwoAttention_Train_Task3_Prompt4_Test_Task3_Prompt4 --pretrain 1 --test 1 --modelid 1 --saved-model ./exp/HSS_BiLSTMwoAttention_Train_Task3_Prompt4/Best_HSS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 9 - Cross Prompt

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 0 HSS_CheckGPTArch_Train_Task1_Prompt1234_Test_Task1_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/HSS_Task1_Prompt1234/Best_LIT_Task1.pth
python dnn.py HSS_10 2 0 HSS_CheckGPTArch_Train_Task2_Prompt1234_Test_Task2_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/HSS_Task2_Prompt1234/Best_LIT_Task2.pth
python dnn.py HSS_10 3 0 HSS_CheckGPTArch_Train_Task3_Prompt1234_Test_Task3_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/HSS_Task3_Prompt1234/Best_LIT_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 234

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 234 HSS_CheckGPTArch_Train_Task1_Prompt234 --batchsize 256
python dnn.py HSS_10 2 234 HSS_CheckGPTArch_Train_Task2_Prompt234 --batchsize 256
python dnn.py HSS_10 3 234 HSS_CheckGPTArch_Train_Task3_Prompt234 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 1 HSS_CheckGPTArch_Train_Task1_Prompt234_Test_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task1_Prompt234/Best_HSS_10_Task1.pth
python dnn.py HSS_10 2 1 HSS_CheckGPTArch_Train_Task2_Prompt234_Test_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task2_Prompt234/Best_HSS_10_Task2.pth
python dnn.py HSS_10 3 1 HSS_CheckGPTArch_Train_Task3_Prompt234_Test_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task3_Prompt234/Best_HSS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 134

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 134 HSS_CheckGPTArch_Train_Task1_Prompt134 --batchsize 256
python dnn.py HSS_10 2 134 HSS_CheckGPTArch_Train_Task2_Prompt134 --batchsize 256
python dnn.py HSS_10 3 134 HSS_CheckGPTArch_Train_Task3_Prompt134 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 2 HSS_CheckGPTArch_Train_Task1_Prompt134_Test_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task1_Prompt134/Best_HSS_10_Task1.pth
python dnn.py HSS_10 2 2 HSS_CheckGPTArch_Train_Task2_Prompt134_Test_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task2_Prompt134/Best_HSS_10_Task2.pth
python dnn.py HSS_10 3 2 HSS_CheckGPTArch_Train_Task3_Prompt134_Test_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task3_Prompt134/Best_HSS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 124

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 124 HSS_CheckGPTArch_Train_Task1_Prompt124 --batchsize 256
python dnn.py HSS_10 2 124 HSS_CheckGPTArch_Train_Task2_Prompt124 --batchsize 256
python dnn.py HSS_10 3 124 HSS_CheckGPTArch_Train_Task3_Prompt124 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 3 HSS_CheckGPTArch_Train_Task1_Prompt124_Test_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task1_Prompt124/Best_HSS_10_Task1.pth
python dnn.py HSS_10 2 3 HSS_CheckGPTArch_Train_Task2_Prompt124_Test_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task2_Prompt124/Best_HSS_10_Task2.pth
python dnn.py HSS_10 3 3 HSS_CheckGPTArch_Train_Task3_Prompt124_Test_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task3_Prompt124/Best_HSS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 123

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 123 HSS_CheckGPTArch_Train_Task1_Prompt123 --batchsize 256
python dnn.py HSS_10 2 123 HSS_CheckGPTArch_Train_Task2_Prompt123 --batchsize 256
python dnn.py HSS_10 3 123 HSS_CheckGPTArch_Train_Task3_Prompt123 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_10 1 4 HSS_CheckGPTArch_Train_Task1_Prompt123_Test_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task1_Prompt123/Best_HSS_10_Task1.pth
python dnn.py HSS_10 2 4 HSS_CheckGPTArch_Train_Task2_Prompt123_Test_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task2_Prompt123/Best_HSS_10_Task2.pth
python dnn.py HSS_10 3 4 HSS_CheckGPTArch_Train_Task3_Prompt123_Test_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/HSS_CheckGPTArch_Train_Task3_Prompt123/Best_HSS_10_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Feature Clean-up

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/
shopt -s extglob
rm -rf embeddings/!(README.md)

## CS,PHX,HSS - 2000

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python features.py CS 1 1 --gpt 0 --number 10
python features.py CS 2 1 --gpt 0 --number 10

python features.py CS 1 1 --gpt 1 --number 10
python features.py CS 1 2 --gpt 1 --number 10
python features.py CS 1 3 --gpt 1 --number 10
python features.py CS 1 4 --gpt 1 --number 10

python features.py CS 2 1 --gpt 1 --number 10
python features.py CS 2 2 --gpt 1 --number 10
python features.py CS 2 3 --gpt 1 --number 10
python features.py CS 2 4 --gpt 1 --number 10

python features.py CS 3 1 --gpt 1 --number 10
python features.py CS 3 2 --gpt 1 --number 10
python features.py CS 3 3 --gpt 1 --number 10
python features.py CS 3 4 --gpt 1 --number 10

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python features.py PHX 1 1 --gpt 0 --number 10
python features.py PHX 2 1 --gpt 0 --number 10

python features.py PHX 1 1 --gpt 1 --number 10
python features.py PHX 1 2 --gpt 1 --number 10
python features.py PHX 1 3 --gpt 1 --number 10
python features.py PHX 1 4 --gpt 1 --number 10

python features.py PHX 2 1 --gpt 1 --number 10
python features.py PHX 2 2 --gpt 1 --number 10
python features.py PHX 2 3 --gpt 1 --number 10
python features.py PHX 2 4 --gpt 1 --number 10

python features.py PHX 3 1 --gpt 1 --number 10
python features.py PHX 3 2 --gpt 1 --number 10
python features.py PHX 3 3 --gpt 1 --number 10
python features.py PHX 3 4 --gpt 1 --number 10

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python features.py HSS 1 1 --gpt 0 --number 10
python features.py HSS 2 1 --gpt 0 --number 10

python features.py HSS 1 1 --gpt 1 --number 10
python features.py HSS 1 2 --gpt 1 --number 10
python features.py HSS 1 3 --gpt 1 --number 10
python features.py HSS 1 4 --gpt 1 --number 10

python features.py HSS 2 1 --gpt 1 --number 10
python features.py HSS 2 2 --gpt 1 --number 10
python features.py HSS 2 3 --gpt 1 --number 10
python features.py HSS 2 4 --gpt 1 --number 10

python features.py HSS 3 1 --gpt 1 --number 10
python features.py HSS 3 2 --gpt 1 --number 10
python features.py HSS 3 3 --gpt 1 --number 10
python features.py HSS 3 4 --gpt 1 --number 10

In [ ]:
elapsed_time(start)
start = now()

### Table 9 - Cross Prompt,Domain

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py ALL_10 1 0 CSPHXHSS_CheckGPTArch_Train_Task1_Prompt1234_Test_Task1_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task1_Prompt1234/Best_ALL_Task1.pth --sample 10
python dnn.py ALL_10 2 0 CSPHXHSS_CheckGPTArch_Train_Task2_Prompt1234_Test_Task2_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task2_Prompt1234/Best_ALL_Task2.pth --sample 10
python dnn.py ALL_10 3 0 CSPHXHSS_CheckGPTArch_Train_Task3_Prompt1234_Test_Task3_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task3_Prompt1234/Best_ALL_Task3.pth --sample 10

In [ ]:
elapsed_time(start)
start = now()

### Table 9 - Cross Prompt,Domain,Task

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py ALL_10 0 0 CSPHXHSS_CheckGPTArch_Train_Task123_Prompt1234_Test_Task123_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task123_Prompt1234/Best_ALL_Task0.pth --sample 10

In [ ]:
elapsed_time(start)
start = now()

### Feature Clean-up

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/
shopt -s extglob
rm -rf embeddings/!(README.md)